In [4]:
# Imporrt necessary libraries.
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import brute, fmin
from scipy.integrate import quad

In [11]:
class Heston_model():
    # iteration for error function
    ite = 0
    def H93_char_func(self,u, T, r, kappa_v, theta_v, sigma_v, rho, v0):
        """Valuation of European call option in H93 model via Lewis (2001)
        Fourier-based approach: characteristic function.
        Parameter definitions see function BCC_call_value."""
        c1 = kappa_v * theta_v
        c2 = -np.sqrt(
            (rho * sigma_v * u * 1j - kappa_v) ** 2 - sigma_v**2 * (-u * 1j - u**2)
        )
        c3 = (kappa_v - rho * sigma_v * u * 1j + c2) / (
            kappa_v - rho * sigma_v * u * 1j - c2
        )
        H1 = r * u * 1j * T + (c1 / sigma_v**2) * (
            (kappa_v - rho * sigma_v * u * 1j + c2) * T
            - 2 * np.log((1 - c3 * np.exp(c2 * T)) / (1 - c3))
        )
        H2 = (
            (kappa_v - rho * sigma_v * u * 1j + c2)
            / sigma_v**2
            * ((1 - np.exp(c2 * T)) / (1 - c3 * np.exp(c2 * T)))
        )
        char_func_value = np.exp(H1 + H2 * v0)
        return char_func_value

    def H93_int_func(self,u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
        """
        Fourier-based approach for Lewis (2001): Integration function.
        """
        char_func_value = self.H93_char_func(
            u - 1j * 0.5, T, r, kappa_v, theta_v, sigma_v, rho, v0
        )
        int_func_value = (
            1 / (u**2 + 0.25) * (np.exp(1j * u * np.log(S0 / K)) * char_func_value).real
        )
        return int_func_value
    
    
    def H93_call_value(self, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
        """Valuation of European call option in H93 model via Lewis (2001)

        Parameter definition:
        ==========
        S0: float
            initial stock/index level
        K: float
            strike price
        T: float
            time-to-maturity (for t=0)
        r: float
            constant risk-free short rate
        kappa_v: float
            mean-reversion factor
        theta_v: float
            long-run mean of variance
        sigma_v: float
            volatility of variance
        rho: float
            correlation between variance and stock/index level
        v0: float
            initial level of variance
        Returns
        =======
        call_value: float
            present value of European call option
        """
        int_value = quad(
            lambda u: self.H93_int_func(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0),
            0,
            np.inf,
            limit=250,
        )[0]
        call_value = max(0, S0 - np.exp(-r * T) * np.sqrt(S0 * K) / np.pi * int_value)
        return call_value

    
    def H93_error_function(self,p0):
        """Error function for parameter calibration via
        Lewis (2001) Fourier approach for Heston (1993).
        Parameters
        ==========
        kappa_v: float
            mean-reversion factor
        theta_v: float
            long-run mean of variance
        sigma_v: float
            volatility of variance
        rho: float
            correlation between variance and stock/index level
        v0: float
            initial, instantaneous variance
        Returns
        =======
        MSE: float
            mean squared error
        """
        min_MSE = 500
        kappa_v, theta_v, sigma_v, rho, v0 = p0
        if kappa_v < 0.0 or theta_v < 0.005 or sigma_v < 0.0 or rho < -1.0 or rho > 1.0:
            return min_MSE
        if 2 * kappa_v * theta_v < sigma_v**2:
            return min_MSE
        se = []
        for row, option in options.iterrows():
            model_value = self.H93_call_value(
                S0,
                option["Strike"],
                option["T"],
                option["r"],
                kappa_v,
                theta_v,
                sigma_v,
                rho,
                v0,
            )
            se.append((model_value - option["Price"]) ** 2)
        MSE = sum(se) / len(se)
        #min_MSE = min(min_MSE, MSE)
        if Heston_model.ite % 25 == 0:
            print("%4d |" % Heston_model.ite, np.array(p0), "| %7.3f" % (MSE))
        Heston_model.ite += 1
        return MSE
    

    def H93_calibration_full(self):
        """Calibrates Heston (1993) stochastic volatility model to market quotes."""
        # First run with brute force
        # (scan sensible regions, for faster convergence)
        p0 = brute(
            self.H93_error_function,
            (
                (2.5, 10.6, 5.0),  # kappa_v
                (0.01, 0.041, 0.01),  # theta_v
                (0.05, 0.251, 0.1),  # sigma_v
                (-0.75, 0.01, 0.25),  # rho
                (0.01, 0.031, 0.01),
            ),  # v0
            finish=None,
        )

        # Second run with local, convex minimization
        # (we dig deeper where promising results)
        opt = fmin(
            self.H93_error_function, p0, xtol=0.000001, ftol=0.000001, maxiter=750, maxfun=900
        )
        return opt

In [1]:
def H93_char_func(u, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    """Valuation of Asian call option in H93 model via Lewis (2001)
    Fourier-based approach: characteristic function."""

    c1 = kappa_v * theta_v
    c2 = -np.sqrt(
        (rho * sigma_v * u * 1j - kappa_v) ** 2 - sigma_v**2 * (-u * 1j - u**2)
    )
    c3 = (kappa_v - rho * sigma_v * u * 1j + c2) / (
        kappa_v - rho * sigma_v * u * 1j - c2
    )
    H1 = r * u * 1j * T + (c1 / sigma_v**2) * (
        (kappa_v - rho * sigma_v * u * 1j + c2) * T
        - 2 * np.log((1 - c3 * np.exp(c2 * T)) / (1 - c3))
    )
    H2 = (
        (kappa_v - rho * sigma_v * u * 1j + c2)
        / sigma_v**2
        * ((1 - np.exp(c2 * T)) / (1 - c3 * np.exp(c2 * T)))
    )
    char_func_value = np.exp(H1 + H2 * v0)
    return char_func_value

def H93_int_func(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    """
    Fourier-based approach for Lewis (2001): Integration function.
    """
    char_func_value = H93_char_func(
        u - 1j * 0.5, T, r, kappa_v, theta_v, sigma_v, rho, v0
    )
    int_func_value = (
        1 / (u**2 + 0.25) * (np.exp(1j * u * np.log(S0 / K)) * char_func_value).real
    )
    return int_func_value


def H93_call_value(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    """Valuation of European call option in H93 model via Lewis (2001)

    Parameter definition:
    ==========
    S0: float
        initial stock/index level
    K: float
        strike price
    T: float
        time-to-maturity (for t=0)
    r: float
        constant risk-free short rate
    kappa_v: float
        mean-reversion factor
    theta_v: float
        long-run mean of variance
    sigma_v: float
        volatility of variance
    rho: float
        correlation between variance and stock/index level
    v0: float
        initial level of variance
    Returns
    =======
    call_value: float
        present value of European call option
    """
    int_value = quad(
        lambda u: H93_int_func(u, S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0),
        0,
        np.inf,
        limit=250,
    )[0]
    call_value = max(0, S0 - np.exp(-r * T) * np.sqrt(S0 * K) / np.pi * int_value)
    return call_value



In [9]:
i = 0
min_MSE = 500

def H93_error_function(p0):
    """Error function for parameter calibration via
    Lewis (2001) Fourier approach for Heston (1993).
    Parameters
    ==========
    kappa_v: float
        mean-reversion factor
    theta_v: float
        long-run mean of variance
    sigma_v: float
        volatility of variance
    rho: float
        correlation between variance and stock/index level
    v0: float
        initial, instantaneous variance
    Returns
    =======
    MSE: float
        mean squared error
    """
    global i, min_MSE
    kappa_v, theta_v, sigma_v, rho, v0 = p0
    if kappa_v < 0.0 or theta_v < 0.005 or sigma_v < 0.0 or rho < -1.0 or rho > 1.0:
        return 500.0
    if 2 * kappa_v * theta_v < sigma_v**2:
        return 500.0
    se = []
    for row, option in options.iterrows():
        model_value = H93_call_value(
            S0,
            option["Strike"],
            option["T"],
            option["r"],
            kappa_v,
            theta_v,
            sigma_v,
            rho,
            v0,
        )
        se.append((model_value - option["Call"]) ** 2)
    MSE = sum(se) / len(se)
    min_MSE = min(min_MSE, MSE)
    if i % 25 == 0:
        print("%4d |" % i, np.array(p0), "| %7.3f | %7.3f" % (MSE, min_MSE))
    i += 1
    return MSE

def H93_calibration_full():
    """Calibrates Heston (1993) stochastic volatility model to market quotes."""
    # First run with brute force
    # (scan sensible regions, for faster convergence)
    p0 = brute(
        H93_error_function,
        (
                (2.5, 10.6, 5.0),  # kappa_v
                (0.01, 0.041, 0.01),  # theta_v
                (0.05, 0.251, 0.1),  # sigma_v
                (-0.75, 0.01, 0.25),  # rho
                (0.01, 0.031, 0.01),
            ),  # v0
        finish=None,
    )

    # Second run with local, convex minimization
    # (we dig deeper where promising results)
    opt = fmin(
        H93_error_function, p0, xtol=0.000001, ftol=0.000001, maxiter=950, maxfun=900
    )
    return opt

In [5]:
# Market Data from www.eurexchange.com
# as of September 30, 2014

h5 = pd.HDFStore(
    "option_data_wqu.h5", "r"
)  # Place this file in the same directory before running the code
data = h5["data"]  # European call & put option data (3 maturities)
h5.close()
S0 = 3225.93  # EURO STOXX 50 level September 30, 2014

In [6]:
# Option Selection
tol = 0.02  # Tolerance level to select ATM options (percent around ITM/OTM options)
options = data[(np.abs(data["Strike"] - S0) / S0) < tol]
options["Date"] = pd.DatetimeIndex(options["Date"])
options["Maturity"] = pd.DatetimeIndex(options["Maturity"])

C:\Users\AzazH\AppData\Local\Temp\ipykernel_3896\114457767.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options["Date"] = pd.DatetimeIndex(options["Date"])
C:\Users\AzazH\AppData\Local\Temp\ipykernel_3896\114457767.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options["Maturity"] = pd.DatetimeIndex(options["Maturity"])


In [7]:
# Adding Time-to-Maturity and constant short-rates

for row, option in options.iterrows():
    T = (option["Maturity"] - option["Date"]).days / 365.0
    options.loc[row, "T"] = T
    options.loc[row, "r"] = 0.02

C:\Users\AzazH\AppData\Local\Temp\ipykernel_3896\1602595275.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options.loc[row, "T"] = T
C:\Users\AzazH\AppData\Local\Temp\ipykernel_3896\1602595275.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options.loc[row, "r"] = 0.02


In [8]:
options.head()

,Date,Strike,Call,Maturity,Put,T,r
38,2014-09-30,3175.0,126.8,2014-12-19,78.8,0.219178,0.02
39,2014-09-30,3200.0,110.9,2014-12-19,87.9,0.219178,0.02
40,2014-09-30,3225.0,96.1,2014-12-19,98.1,0.219178,0.02
41,2014-09-30,3250.0,82.3,2014-12-19,109.3,0.219178,0.02
42,2014-09-30,3275.0,69.6,2014-12-19,121.6,0.219178,0.02


In [10]:
H93_calibration_full()

   0 | [ 2.5   0.01  0.05 -0.75  0.01] | 820.892 | 500.000
  25 | [ 2.5   0.02  0.05 -0.75  0.02] |  23.864 |  21.568
  50 | [ 2.5   0.02  0.25 -0.75  0.03] |  89.655 |  21.568
  75 | [ 2.5   0.03  0.15 -0.5   0.01] | 193.283 |  21.568
 100 | [ 2.5   0.04  0.05 -0.5   0.02] | 176.340 |  21.568
 125 | [ 2.5   0.04  0.25 -0.5   0.03] | 486.965 |  21.568
 150 | [ 7.5   0.01  0.15 -0.25  0.01] | 840.337 |  21.568
 175 | [ 7.5   0.02  0.05 -0.25  0.02] |  24.810 |  21.568
 200 | [ 7.5   0.02  0.25 -0.25  0.03] |  24.834 |  21.568
 225 | [7.5  0.03 0.15 0.   0.01] | 110.936 |  21.568
 250 | [7.5  0.04 0.05 0.   0.02] | 540.183 |  21.568
 275 | [7.5  0.04 0.25 0.   0.03] | 783.222 |  21.568
 300 | [ 2.61379559  0.00992657  0.15610448 -0.76361614  0.02778356] |   8.046 |   7.795
 325 | [ 1.9152359   0.01257942  0.16036675 -0.91693167  0.0248233 ] |   6.301 |   6.146
 350 | [ 2.04831069  0.01215428  0.15832201 -0.89057611  0.02532865] |   6.151 |   6.145
 375 | [ 2.0376908   0.01207312  0.16816

array([ 5.04735825,  0.01872573,  0.43477689, -0.44795621,  0.02728912])

In [11]:
#importing market data

# Given 
S0 = 232.9
r=1.5/100

options = pd.read_excel("MScFE 622_Stochastic Modeling_GWP1_Option data.xlsx" )
options["T"] = options["Days to maturity"]/250.0
options["r"] = r

options.head()

,Days to maturity,Strike,Price,Type,T,r
0,15,227.5,10.52,C,0.06,0.015
1,15,230.0,10.05,C,0.06,0.015
2,15,232.5,7.75,C,0.06,0.015
3,15,235.0,6.01,C,0.06,0.015
4,15,237.5,4.75,C,0.06,0.015
